In [1]:
import tensorflow as tf

from tensorflow.keras.layers import Flatten, Input, Dense

class Generator(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(128, activation='relu')
        self.dense3 = Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)

class Discriminator(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = Dense(128, activation='relu')
        self.dense2 = Dense(128, activation='relu')
        self.dense3 = Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        return self.dense3(x)

generator = Generator()
discriminator = Discriminator()

optimizer_gen = tf.keras.optimizers.Adam(learning_rate=0.0001)
optimizer_disc = tf.keras.optimizers.Adam(learning_rate=0.0001)

epochs = 100

for epoch in range(epochs):
    # Train the discriminator
    noise = tf.random.normal([100, 10])
    generated_data = generator(noise)
    real_data = tf.random.uniform([100, 10], minval=0, maxval=10, dtype=tf.int32)

    with tf.GradientTape() as tape:
        d_loss_real = discriminator(real_data)
        d_loss_fake = discriminator(generated_data)
        d_loss = tf.reduce_mean(d_loss_real) - tf.reduce_mean(d_loss_fake)

    optimizer_disc.minimize(d_loss,var_list=discriminator.trainable_variables, tape=tape)

    # Train the generator
    noise = tf.random.normal([100, 10])
    generated_data = generator(noise)

    with tf.GradientTape() as tape:
        g_loss = tf.keras.losses.binary_crossentropy(real_data, discriminator(generated_data))

    optimizer_gen.minimize(g_loss,  var_list=generator.trainable_variables, tape=tape)

    print(f"Epoch {epoch}: d_loss={d_loss}, g_loss={g_loss}")


2023-07-13 18:37:00.161842: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/namachu/.local/lib/python3.11/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/home/namachu/.local/lib/python3.11/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/home/namachu/.local/lib/python3.11/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6Status12empty_stringB5cxx11Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/home/namachu/.local/lib/python3.11/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not 

ValueError: `logits` and `labels` must have the same shape, received ((100, 1) vs (100, 10)).